# Music genre classification

## Dataset description

This dataset wad download from kaggle.com. According to the description, it was created as a part of MachineHack Hackathon. [Link to data](https://www.kaggle.com/datasets/purumalgi/music-genre-classification?select=test.csv)

### Features

The dataset has 17 features altogether.

Following table contains dataset feature names, their types and short description:

| Feature name | Type | Description |
| ---- | ---- | ---- |
| Artist Name | string | Artist name |
| Track Name | string | Name of the track |
| Popularity | integer | Measures how the song is popular |
| danceability | float | Measures how the song is danceable  |
| energy | float | Measure how energetic the song is |
| key | categorical | Determines the key of the song |
| loudness | float | Measures how loud the song is |
| mode | categorical | Determines the song's mode |
| speechiness | float | Measures how lyrical the song is |
| acousticness | float | Measure how acoustic the song is |
| instrumentalness | float | Measure how instrumental the song is |
| liveness | float | Measures how lively the song is |
| valence | float | Measures how positive the song is|
| tempo | float | Measures song tempo |
| duration_in min/ms | float | Song duration |
| time_signature | categorical | Determines time signature |
| Class | categorical | Determines which genre the song belongs to |


### Target

The aim of this dataset (and of this notebook) is to train a model to predict the genre a song belongs to. Following table provides overview of genres present in the dataset:

| Class in dataset | Genre |
| ---- | ---- |
| 0 | Acoustic |
| 1 | Alt |
| 2 | Blues |
| 3 | Bollywood |
| 4 | Country |
| 5 | HipHop |
| 6 | Indie |
| 7 | Instrumental |
| 8 | Metal |
| 9 | Pop |
| 10 | Rock |


In [1]:
# import libraries and functions for the whole notebook
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# set seed for reproducible results - sklearn follows this seed as well
np.random.seed(565)

## Creating training and testing dataset.

The data provided on kaggle does not have the target variable present in the testing dataset, so the training dataset will be used for the purposes of training and testing. The dataset will be read and split into training and testing. The split will be done in a stratified fashion to preserve the genre distribution between test and train data.

The exploratory analysis and preprocessing will be done only with the training part in order to avoid data leakage from the testing part.

In [2]:
data = pd.read_csv("data/train.csv", dtype={"key": "category", "mode": "category", "time_signature": "category", "Class": "category"})

train, test = train_test_split(data, test_size=0.3, stratify=data["Class"])

# show automatically detected dtypes (will be fixed later)
data.dtypes

Artist Name             object
Track Name              object
Popularity             float64
danceability           float64
energy                 float64
key                   category
loudness               float64
mode                  category
speechiness            float64
acousticness           float64
instrumentalness       float64
liveness               float64
valence                float64
tempo                  float64
duration_in min/ms     float64
time_signature        category
Class                 category
dtype: object

## Exploratory data analysis

In this section, we will look at how values are distributed in the dataset and how they're individually correlated to the output.

### Missing values

However, first a quick look on missing values in the dataset. It will give overview if certain features can or should be discarded. 

#### Missing values in columns

As it can be seen, only three features have at least one missing value. Result shows percentage of rows in each column that has a missing value. Feature `instrumentalness` is missing a quarter of its values and `key` has 11% missing. Those two features will be discarded due to high amount of missing values. `Popularity` has only 2% values missing, this value is acceptable.

In [3]:
train.isna().sum(axis=0)/train.shape[0] * 100

Artist Name            0.000000
Track Name             0.000000
Popularity             2.413273
danceability           0.000000
energy                 0.000000
key                   11.209018
loudness               0.000000
mode                   0.000000
speechiness            0.000000
acousticness           0.000000
instrumentalness      24.339128
liveness               0.000000
valence                0.000000
tempo                  0.000000
duration_in min/ms     0.000000
time_signature         0.000000
Class                  0.000000
dtype: float64

#### Missing values in rows

34% of songs have at least 1 feature missing. With the two features being dropped however, the percentage would drop to 2.2%.

In [4]:
(train.isna().sum(axis=1)>0).sum()/train.shape[0]*100

34.325633087243

### Artist name

Artist names are provided as strings. Processing these could be difficult as there are 7286 unique artists in the dataset. There are also some artists represented in multiple unique strings because they can be part of a collaboration between multiple artists. 

Top 10 artists cover 2.56% of all songs. It could be worth to try to include one-hot encoded top N artists as an experiment. Including high number of artists will lead to high cardinality, and the model will probably not be able to capture the relationship for those artists that have just a few songs represented in this dataset. 

In [5]:
print(f"Amount of unique artist strings: {len(train['Artist Name'].unique())}")

top_artists = 10

print(f"Top {top_artists} artists cover {train['Artist Name'].value_counts()[:top_artists].sum()/train.shape[0] * 100:.2f}% of all training data.")

Amount of unique artist strings: 7226
Top 10 artists cover 2.51% of all training data.


### Track name

As can be seen in the output below, there are some track names being represented multiple times. However, these numbers are very low to be significant enough to be used as a variable.

In [6]:
train['Track Name'].value_counts()

Hurricane                                                          7
Fire                                                               6
Dreams                                                             6
Ride                                                               5
Over You                                                           5
                                                                  ..
World On Silent                                                    1
Wind Parade                                                        1
Are the Good Times Really Over (I Wish a Buck Was Still Silver)    1
Love Can Pass You By                                               1
◊í◊ú◊í◊ú ◊¢◊†◊ß                                                    1
Name: Track Name, Length: 11065, dtype: int64

### Popularity

In [7]:
#Removing the variables for 'key' and 'instrumentalness' due to the high percentage of missing values

data_without_missing_values = train.drop(['key', 'instrumentalness'], axis=1)
data_without_missing_values.head()

,Artist Name,Track Name,Popularity,danceability,energy,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
6368,Skip James,Cypress Grove Blues,26.0,0.599,0.121,-18.501,1,0.0583,0.9920,0.0933,0.568,102.598,189187.0,4,2
16126,Leah Kate,Boyfriend,52.0,0.898,0.509,-6.038,0,0.1240,0.1580,0.0939,0.555,108.002,143556.0,4,9
8992,ONUKA,ZENIT,46.0,0.658,0.679,-5.766,1,0.0445,0.1070,0.3890,0.247,140.055,271189.0,4,6
4415,Electroforez,–ê–ª–∫–æ–≥–æ–ª—å –º–æ–π –≤—Ä–∞–≥,46.0,0.682,0.877,-4.888,0,0.0410,0.0309,0.0847,0.827,111.964,167000.0,4,6
1457,Grimes,Violence - REZZ Remix,59.0,0.596,0.661,-7.533,0,0.0382,0.0713,0.2090,0.124,99.951,195300.0,4,10


In [8]:
#Removing 'Artist Name' and 'Track Name' due to their unique values and non-essential variables in determing the music genre

data_without_missing_and_redundant_values = data_without_missing_values.drop(['Artist Name', 'Track Name'], axis=1)
data_without_missing_and_redundant_values.head()

,Popularity,danceability,energy,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
6368,26.0,0.599,0.121,-18.501,1,0.0583,0.9920,0.0933,0.568,102.598,189187.0,4,2
16126,52.0,0.898,0.509,-6.038,0,0.1240,0.1580,0.0939,0.555,108.002,143556.0,4,9
8992,46.0,0.658,0.679,-5.766,1,0.0445,0.1070,0.3890,0.247,140.055,271189.0,4,6
4415,46.0,0.682,0.877,-4.888,0,0.0410,0.0309,0.0847,0.827,111.964,167000.0,4,6
1457,59.0,0.596,0.661,-7.533,0,0.0382,0.0713,0.2090,0.124,99.951,195300.0,4,10


In [9]:
(data_without_missing_and_redundant_values.isna().sum(axis=1)>0).sum()/data_without_missing_and_redundant_values.shape[0]*100

2.413273001508296

In [10]:
#Missing values in popularity variable are replaced using median() as animputation technique. 
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median', missing_values=np.nan)
imputer = imputer.fit(data_without_missing_and_redundant_values[['Popularity']])
data_without_missing_and_redundant_values['Popularity'] = imputer.transform(data_without_missing_and_redundant_values[['Popularity']])
data_without_missing_and_redundant_values.head()

,Popularity,danceability,energy,loudness,mode,speechiness,acousticness,liveness,valence,tempo,duration_in min/ms,time_signature,Class
6368,26.0,0.599,0.121,-18.501,1,0.0583,0.9920,0.0933,0.568,102.598,189187.0,4,2
16126,52.0,0.898,0.509,-6.038,0,0.1240,0.1580,0.0939,0.555,108.002,143556.0,4,9
8992,46.0,0.658,0.679,-5.766,1,0.0445,0.1070,0.3890,0.247,140.055,271189.0,4,6
4415,46.0,0.682,0.877,-4.888,0,0.0410,0.0309,0.0847,0.827,111.964,167000.0,4,6
1457,59.0,0.596,0.661,-7.533,0,0.0382,0.0713,0.2090,0.124,99.951,195300.0,4,10


## Preprocessing

TODO

> Requirements on preprocessing
> 
> Any two of the following operations are mandatory:
> 
> -     remove rows based on subsetting
> -     derive new columns
> -     use aggregation operators
> -     treat missing values


TODO remove instrumentalness


## Modeling

TODO

> Requirements
> 
> Use any classifier. Choose one of the following two options:
> 
> -     perform train/test split
> -     use crossvalidation
> 
> Also, evaluate and compare at least two algorithms of different types (e.g. logistic regression and random forest).
> Any classifier from `sklearn`

### Hyperparameter tuning

TODO

> Requirements on metaparameter tuning
> 
> If the chosen classifier has any metaparameters that can be tuned, use one of the following methods:
> 
>  -   try several configurations and describe the best result in the final report
>  -   perform grid search or other similar automatic method
>  -   once you have tuned metaparameters on a dedicated development (training) set, e.g. with GridSearchCV, you can retrain the model on the complete training data, as e.g. described here for Python: https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html and https://stackoverflow.com/questions/26962050/confused-with-repect-to-working-of-gridsearchcv
> 
> Python recommendation: sklearn.model_selection.GridSearchCV

## Results and evaluation

TODO

> Requirements on model evaluation
> 
> -    report the accuracy on test set/crossvalidation
> -    if you are performing binary classification task, involve also the ROC curve
> -    make sure to use dedicated dataset for evaluation
> 
> Python: use model_selection.cross_val_score, plot the roc curve using sklearn.metrics.roc_curve R: print model learned using the caret package, the roc curve can be plotted using the plotROC package.